In [5]:
import os
import time
import logging
from sqlalchemy import create_engine
import pandas as pd  

In [6]:
os.makedirs("logs", exist_ok=True)
os.makedirs("data", exist_ok=True)  

In [7]:
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)


In [8]:
engine = create_engine('sqlite:///inventory.db')


In [9]:
def ingest_db(df, table_name, engine):
    """
    Ingests the dataframe into the specified database table.
    Parameters:
        df (DataFrame): The data to ingest
        table_name (str): Name of the target table
        engine (SQLAlchemy Engine): Database connection
    """
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)


In [10]:
def load_raw_data():
    """
    Loads CSV files from the 'data' folder and ingests them into the database.
    Logs progress and errors.
    """
    start = time.time()

    for file in os.listdir('data'):
        if file.endswith('.csv'):
            try:
                file_path = os.path.join('data', file)
                df = pd.read_csv(file_path)

                logging.info(f'Ingesting {file} into database')
                ingest_db(df, file[:-4], engine)

            except Exception as e:
                logging.error(f'Error processing {file}: {e}')

    end = time.time()
    total_time = (end - start) / 60

    logging.info('--------------Ingestion Complete------------')
    logging.info(f'Total Time Taken: {total_time:.2f} minutes')


In [11]:
if __name__ == '__main__':
    load_raw_data()
